In [1]:
val i: Int = 3.5

cmd1.sc:1: type mismatch;
 found   : Double(3.5)
 required: Int
val i: Int = 3.5
             ^

: 

In [2]:
implicit def double2Int(d: Double) = d.toInt

val i: Int = 3.5

defined function double2Int
i: Int = 3

In [3]:
val d: Double = 3

d: Double = 3.0

In [4]:
case class Rational(n: Int, d: Int) {
    def +(other: Rational) = Rational(n + other.n, d + other.d)
    def +(other: Int) = Rational(n + other, d)
}

defined class Rational

In [5]:
val oneHalf = Rational(1, 2)

oneHalf + oneHalf
oneHalf + 1

oneHalf: Rational = Rational(1, 2)
res4_1: Rational = Rational(2, 4)
res4_2: Rational = Rational(2, 2)

In [6]:
implicit def intToRational(x: Int) = Rational(x, 0)

defined function intToRational

In [7]:
1 + oneHalf

res6: Rational = Rational(2, 2)

In [8]:
val map = Map(1 -> "one", 2 -> "two", 3 -> "three")

map: Map[Int, String] = Map(1 -> "one", 2 -> "two", 3 -> "three")

In [9]:
val tu = Tuple2(1, 3)

tu: (Int, Int) = (1, 3)

In [10]:
object Helpers {
    implicit class IntWithTimes(x: Int) {
        def times[A](f: => A): Unit = {
            def loop(current: Int): Unit = 
                if(current > 0) {
                    f
                    loop(current - 1)
                }
            loop(x)
        }
    }
}

defined object Helpers

In [11]:
import Helpers._
5 times println("Hi")

Hi
Hi
Hi
Hi
Hi


import Helpers._


In [12]:
def calcTax(amount: Float)(implicit rate: Float): Float =
    amount * rate

defined function calcTax

In [13]:
implicit val currentTaxRate = 0.08F

currentTaxRate: Float = 0.08F

In [14]:
val tax = calcTax(50000)

tax: Float = 4000.0F

In [15]:
def maxList[T](elements: List[T])(implicit ordering: Ordering[T]): T = 
    elements match {
        case List() => throw new IllegalArgumentException("empty list!")
        case List(x) => x
        case x :: rest => 
            val maxRest = maxList(rest)
            if (ordering.gt(x, maxRest)) x
            else maxRest
    }

defined function maxList

In [16]:
def maxList[T : Ordering](elements: List[T]): T = elements match {
    case List() => throw new IllegalArgumentException("empty list!")
    case List(x) => x
    case x :: rest => 
        val maxRest = maxList(rest)
        if (implicitly[Ordering[T]].gt(x, maxRest)) x
        else maxRest
}

defined function maxList

In [17]:
maxList(List(3,2,4,1,2,5))

res16: Int = 5

In [18]:
case class Address(street: String, city: String)
case class Person(name: String, address: Address)

defined class Address
defined class Person

In [19]:
val address = Address("15th duff", "Ames")
val person = Person("Peter", address)

address: Address = Address("15th duff", "Ames")
person: Person = Person(Peter,Address(15th duff,Ames))

In [19]:
println(address.toJSON())

cmd19.sc:1: value toJSON is not a member of cmd19Wrapper.this.cmd18.cmd17.wrapper.Address
val res19 = println(address.toJSON())
                            ^

: 

In [20]:
trait ToJSON {
    def toJSON(level: Int = 0): String
    
    val INDENTATION = " "
    def indentation(level: Int = 0): (String, String) =
        (INDENTATION * level, INDENTATION * (level + 1))
}

defined trait ToJSON

In [21]:
object DefaultImplicits {
    implicit class AddressToJSON(address: Address) extends ToJSON {
        def toJSON(level: Int = 0): String = {
            val (outdent, indent) = indentation(level)
            s"""{
            |${indent}"street": "${address.street}",
            |${indent}"city": "${address.city}"
            |${outdent}} """.stripMargin
        }
    }
}

defined object DefaultImplicits

In [22]:
import DefaultImplicits._
println(address.toJSON())

{
 "street": "15th duff",
 "city": "Ames"
} 


import DefaultImplicits._


# Type Class

In [23]:
trait InfoPrinter[A] {
    def toInfo(value: A): String
}

defined trait InfoPrinter

## Type Class Instances

In [24]:
object InfoPrinterInstances {
    implicit val stringPrinter = new InfoPrinter[String] {
        def toInfo(value: String): String = s"[String] $value"
    }
    implicit val intPrinter = new InfoPrinter[Int] {
        def toInfo(value: Int): String = s"[Int] $value"
    }
}

defined object InfoPrinterInstances

In [25]:
import InfoPrinterInstances._

import InfoPrinterInstances._

## Interface Objects

In [26]:
object PrintInfo {
    def toInfo[A](value: A)(implicit printer: InfoPrinter[A]): String =
        printer.toInfo(value)
}

defined object PrintInfo

In [27]:
val msg = 3
val result = PrintInfo.toInfo(msg)

msg: Int = 3
result: String = "[Int] 3"

## Interface syntax

In [28]:
object PrintInfoSyntax {
    implicit class PrintInfoOps[A](value: A) {
        def toInfo(implicit printer: InfoPrinter[A]): String =
            printer.toInfo(value)
    }
}

defined object PrintInfoSyntax

In [29]:
import PrintInfoSyntax._

import PrintInfoSyntax._

In [30]:
msg.toInfo

res29: String = "[Int] 3"

In [31]:
case class User(name: String, age: Int)

object User {
    implicit val userPrinter = new InfoPrinter[User] {
        override def toInfo(value: User): String =
            s"[User] (${value.name}, ${value.age})"
    }
}

defined class User
defined object User

In [32]:
import User._

import User._

In [33]:
val user = User("Peter", 18)

user: User = User("Peter", 18)

In [34]:
user.toInfo

res33: String = "[User] (Peter, 18)"

## Another Variation (old style)

In [36]:
trait PrintInfoOps[A] {
    def a: A
    implicit def F: InfoPrinter[A]
    
    def toInfo: String = F.toInfo(a)
}

object PrintInfoSyntax {
    import scala.language.implicitConversions
    
    implicit def toPrintInfoOps[A : InfoPrinter](value: A): 
        PrintInfoOps[A] = new PrintInfoOps[A] {
            val a: A = value
            val F: InfoPrinter[A] = implicitly
        }
}

defined trait PrintInfoOps
defined object PrintInfoSyntax

## Example: Disambiguating

In [37]:
case class Person(name: String, age: Int)
val people = List(Person("Harry", 30), Person("Sally", 25))

import scala.math.Ordering
implicit val byName: Ordering[Person] = Ordering.by(_.name)

people.sorted

defined class Person
people: List[wrapper.wrapper.Person] = List(Person("Harry", 30), Person("Sally", 25))
import scala.math.Ordering

byName: Ordering[wrapper.wrapper.Person] = scala.math.Ordering$$anon$9@c9005e1
res36_4: List[wrapper.wrapper.Person] = List(Person("Harry", 30), Person("Sally", 25))

In [38]:
implicit val byAge: Ordering[Person] = Ordering.by(_.age)
// people.sorted    // -> Error! ambiguo

byAge: Ordering[Person] = scala.math.Ordering$$anon$9@b99e74a

In [39]:
people.sorted(byName)
people.sorted(byAge)

res38_0: List[Person] = List(Person("Harry", 30), Person("Sally", 25))
res38_1: List[Person] = List(Person("Sally", 25), Person("Harry", 30))